In [1]:
import gmsh 
from CompositeSandwich.box_labeling import box_labeling
import shutil,os
from djccx.inp.inp import inp
from Composite.inp.CreateNsetFromElset import CreateNsetFromElset


In [2]:

gmsh.initialize()

# Un cubo partido en tres partes
# Para ello haremos primero un cubo y luego un cubo más pequeño en el centro
# luego con la herramienta de partición de gmsh dividiremos el cubo en tres partes
# ademas etiquetaremos las tres partes con physical groups como top, middle y bottom

cube = gmsh.model.occ.addBox(0, 0, 0, 1, 1, 1)

# Cubo más pequeño
small_cube = gmsh.model.occ.addBox(0.0, 0.0, 0.25, 
                                   1.0, 1.0, 0.50)

# Partición

gmsh.model.occ.synchronize()


r = gmsh.model.occ.cut([(3, cube)], [(3, small_cube)], removeTool=False)
gmsh.model.occ.synchronize()

# Etiquetado
# Definimos los grupos físicos

middle = gmsh.model.addPhysicalGroup(3, [small_cube])
gmsh.model.setPhysicalName(3, middle, "middle")

top_tag = r[0][1][1]
bot_tag = r[0][0][1]

top_ph = gmsh.model.addPhysicalGroup(3, [top_tag])
gmsh.model.setPhysicalName(3, top_ph, "top")

bot_ph = gmsh.model.addPhysicalGroup(3, [bot_tag])
gmsh.model.setPhysicalName(3, bot_ph, "bot")

# Box labeling 
box_labeling(bot_tag,"bot") 
box_labeling(top_tag,"top")

gmsh.model.occ.synchronize()

gmsh.model.mesh.generate(3)
# second order
gmsh.model.mesh.setOrder(2)
gmsh.write("cube.inp")
gmsh.finalize()

In [3]:

inp_f = inp("cube.inp")

# --------------------------------
# Creamos los nsets a partir de los elsets
# --------------------------------
bot_surf_elset = inp_f.select("BOT_Z0","elset") # Este viene de la función box_labeling
bot_nset= CreateNsetFromElset(inp_f, bot_surf_elset, "bot")
 
top_surf_elset = inp_f.select("TOP_ZL","elset") # Este viene de la función box_labeling
top_nset= CreateNsetFromElset(inp_f, top_surf_elset, "top")

mat_m = inp_f.CreateElasticMaterial("MAT_1", 1e6, 0.3)
mat_c = inp_f.CreateElasticMaterial("MAT_2", 2e6, 0.3)

elset_bot = inp_f.select("BOT","elset")
elset_top = inp_f.select("TOP","elset")
elset_mid = inp_f.select("MIDDLE","elset")

# remove 2d elements
# Solo usamos los elementos 2D para crear los nsets
# ya no los necesitamos
inp_f.remove_by_type(2)

# --------------------------------
# Section
# --------------------------------
inp_f.CreateSolidSection(elset_bot,mat_m)
inp_f.CreateSolidSection(elset_top,mat_m)

inp_f.CreateSolidSection(elset_mid,mat_c)

# --------------------------------
# Step 
# --------------------------------
istep = inp_f.CreateStaticStep()
istep.CreateBoundary(top_nset,3,0.1)
istep.CreateBoundary(bot_nset,3,0.0)

# remove output folder

shutil.rmtree("output", ignore_errors=True)
os.mkdir("output")

inp_f.run("output")

Running Calculix at:  output
Output file:  c:\Users\djoroya\Documents\GitHub\CITISENS_Composite\scripts\Homogenization_blocks_02\output\out.txt
Command:
 mpiexec -n 4 c:\Users\djoroya\Documents\GitHub\CITISENS_Composite\.conda\Lib\site-packages\djccx\bin\ccx_dynamic.exe main
Error reading cvf file

pid:  8228 

Calculix finished



{'data':       node       x       y       z        D1        D2        D3        SXX  \
 node                                                                          
 1        1  0.0000  0.0000  0.7500  0.009911  0.013307  0.070747   -476.069   
 2        2  0.0000  0.0000  0.2500  0.009902  0.013317  0.029258   -439.816   
 3        3  0.0000  1.0000  0.7500  0.009825 -0.014314  0.070698   -590.123   
 4        4  0.0000  1.0000  0.2500  0.009817 -0.014313  0.029297   -383.584   
 5        5  1.0000  0.0000  0.7500 -0.017729  0.013378  0.070724   -645.865   
 ...    ...     ...     ...     ...       ...       ...       ...        ...   
 2367  2367  0.0606  0.0606  0.9375  0.011839  0.015264  0.092293    224.534   
 2368  2368  0.2108  0.5088  0.8750  0.005939 -0.000768  0.083143   7523.460   
 2369  2369  0.6440  0.5668  0.8750 -0.008710 -0.002648  0.083153  11056.200   
 2370  2370  0.7843  0.3500  0.8750 -0.013566  0.004462  0.083086   8474.040   
 2371  2371  0.6424  0.4862  0.8

In [4]:
inp_f.print("cube_with_nset.inp")